In [12]:
import json
import csv
from tqdm import tqdm_notebook as tqdm

In [16]:
words = []
dataset = []
with open("../datasets/dataset.csv", "r") as f:
    tags = csv.reader(f)
    header = next(tags)
    
    for t in tags:
        words += t
        dataset += [t]

In [6]:
len(set(words))

66

In [13]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import re
import time

In [10]:
# バージョンを表すテキストを削除
def eraseVersion(w):
    w = re.sub('(\d+(?:\.\d+)?)$', '', w)
    return w

In [77]:
# Qiitaからタグ検索
def tagSearch(w, page="1"):
    print("tag:{} Search...page{}".format(w, page))
    options = Options()
    options.add_argument('--headless')
    driver = webdriver.Chrome(chrome_options=options)
    driver.get('https://qiita.com/search?page='+ page +'&sort=like&q=tag%3A' + w)
    data = driver.page_source
    html = BeautifulSoup(data, 'lxml')
    driver.close()
    driver.quit()
    return html

In [8]:
res_d = dict()

for w in tqdm(set(words)):
    res = []
    for p in range(1,6):
        for i in tagSearch(w, page=str(p)).find_all(class_="tagList"):
            res += [[j.text.lower() for j in i.find_all("a")]]
        time.sleep(0.5)
    
    res_d[w] = res

In [9]:
res_d.keys()

dict_keys(['rbenv', 'html', 'kivy', 'next.js', 'javascript', 'npm', 'rstudio', 'scikit-learn', 'nuxt.js', 'jupyter', 'webgl', 'coffeescript', 'pyenv', 'tkinter', 'scipy', 'typescript', 'flask', 'reactnative', 'spring', 'rails', 'redux', 'scss', 'angular', 'backbone.js', 'javafx', 'php', 'numpy', 'symfony', 'd3.js', 'ajax', 'css', 'pyqt', 'chainer', 'django', 'bootstrap', 'eslint', 'vuex', 'yarn', 'node.js', 'chrome-extension', 'ecmascript', 'r', 'beautifulsoup', 'babel', 'jquery', 'three.js', 'vuetify', 'mruby', 'python', 'promise', 'phina.js', 'vue.js', 'pytorch', 'webpack', 'sass', 'electron', 'react', 'capybara', 'keras', 'canvas', 'cakephp', 'java', 'laravel', 'rspec', 'pip', 'ruby'])

In [3]:
with open('qiita_results_data.json', 'w') as f:
    json.dump(res_d, f)

In [4]:
with open('qiita_results_data.json', 'r') as f:
    read_res_d = json.load(f)

In [5]:
read_res_d.keys()

dict_keys(['tkinter', 'electron', 'npm', 'symfony', 'ecmascript', 'laravel', 'scss', 'php', 'angular', 'css', 'typescript', 'html', 'rbenv', 'eslint', 'r', 'ajax', 'vue.js', 'backbone.js', 'bootstrap', 'cakephp', 'phina.js', 'capybara', 'chrome-extension', 'promise', 'reactnative', 'spring', 'yarn', 'chainer', 'scikit-learn', 'beautifulsoup', 'sass', 'three.js', 'pyenv', 'vuex', 'flask', 'babel', 'jupyter', 'rails', 'react', 'rstudio', 'pip', 'next.js', 'd3.js', 'webpack', 'numpy', 'rspec', 'django', 'keras', 'scipy', 'ruby', 'pytorch', 'canvas', 'mruby', 'node.js', 'jquery', 'python', 'vuetify', 'kivy', 'java', 'javascript', 'javafx', 'coffeescript', 'nuxt.js', 'redux', 'pyqt', 'webgl'])

In [6]:
read_res_d["react"]

[['react', 'jquery'],
 ['virtualdom', 'adventcalendar', 'javascript', 'react'],
 ['react', 'redux', 'javascript', 'redux-saga'],
 ['javascript', 'react', 'angular'],
 ['react'],
 ['react', 'css', 'materialdesign', 'ui'],
 ['フロントエンド', 'javascript', 'react', 'プログラミング', 'ソフトウェア開発'],
 ['javascript', 'react'],
 ['react', 'redux', 'material-ui', 'react-router', 'spa'],
 ['vue.js', 'javascript', 'react'],
 ['swift', 'android', 'javascript', 'rails', 'react'],
 ['react', 'redux', 'react-redux', '新人プログラマ応援'],
 ['vue.js', 'react', 'angular2', 'elm', 'architecture'],
 ['javascript', 'react'],
 ['redux', 'redux-saga', 'react'],
 ['javascript', 'es6', 'node.js', 'react', 'angular'],
 ['riot', 'javascript', 'react', 'angular'],
 ['javascript', 'redux', 'react'],
 ['react', 'jquery', 'kotlin', '日本語訳', 'vue.js'],
 ['react', 'babel', 'webpack'],
 ['javascript', 'jest', 'react'],
 ['react'],
 ['react'],
 ['react', 'javascript', 'react16', 'webpack', 'webpack4'],
 ['javascript', 'riot', 'vue.js', 'react'

In [7]:
# w2に対するw1のスコアを返す
def check(w1, w2):
    w1 = w1.lower()
    w2 = w2.lower()
    res = []
    score = 0
    count = 0
        
    # w1で検索した際にw2が含まれていたら1 それ以外は0
    for i in read_res_d[w1]:
        res += [[eraseVersion(j.lower())==w2 for j in i].count(True)]
        count += 1
    time.sleep(0.1)

    # 検索結果が存在しない場合スコア0を返す
    if len(res) == 0:
        return 0

    # w1で検索した時にw2が含まれる割合
    score += ((len(res)-res.count(0))+1e-7)/(len(res)+1e-7)

    # 1つのページの出現率が30%以下ならスコア0を返す
    #if score/p < 0.2:
    #    return 0
    
    #print(len(res)-res.count(0))
        
    return round(score, 3)

In [18]:
# 2つの単語を比較
def comparisonCheck(w1, w2, boundary=0.2, p=True):
    w1_score = check(w1, w2)
    w1, w2 = w2, w1
    w2_score = check(w1, w2)
    w1, w2 = w2, w1
    
    if p:
        print(w1, "Dscore:", w1_score)
        print(w2, "Dscore:", w2_score)
        
    if (w1_score < boundary and w2_score < boundary) or (abs(w1_score-w2_score) < 0.1):
        if p:
            print(w1, "and", w2, "are irrelevant")
        return {}
    else:
        if w1_score < w2_score:
            if p:
                print(">>", w1, "is easy!")
            return {"easy": w1, "hard": w2, "score": max(w1_score, w2_score)}
        else:
            if p:
                print(">>", w2, "is easy!")
            return {"easy": w2, "hard": w1, "score": max(w1_score, w2_score)}

In [19]:
comparisonCheck("javascript", "react")

javascript Dscore: 0.08
react Dscore: 0.62
>> javascript is easy!


{'easy': 'javascript', 'hard': 'react', 'score': 0.62}

In [68]:
len(dataset)

60

In [21]:
correct = 0
check_res = []

for ds in tqdm(dataset):
    
    cr = comparisonCheck(ds[0], ds[1], p=False)
    if cr:
        if [cr["easy"], cr["hard"]] == [ds[0], ds[1]]:
            correct += 1
        else:
            print([ds[0], ds[1]])
    else:
        print([ds[0], ds[1]])
            
print(correct/len(dataset))

['html', 'electron']
['html', 'css']

0.9666666666666667
